<a href="https://colab.research.google.com/github/chiardy90/Titanic---Machine-Learning-from-Disaster/blob/main/Titanic_RF_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic - Machine Learning from Disaster

## Read file

In [80]:
import numpy as np # linear algebra
import pandas as pd

train_data = pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")
train_data.head()  #shape(891,12)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


##Clear data

In [81]:
#Because from the data, it is difficult to think about a relationship between the ID、Ticket、Name and the survival probability.

dropcolumns=['PassengerId','Ticket','Name']
clean_train_data=train_data.drop(columns=dropcolumns)
clean_test_data=test_data.drop(columns=dropcolumns)

clean_train_data.isna().sum().sum()

866

In [82]:
#But age is such a important factor in the survival rate.
#so we delete the rows that has this missing data.first

cont=0
conts=[]
for i in clean_train_data['Age']:
    if train_data['Age'].isna()[cont]==True: #Check the number of NAN
        clean_train_data=clean_train_data.drop(cont,axis=0)
        conts.append(cont)
    cont+=1
       
clean_train_data.isna().sum()
clean_train_data=clean_train_data.drop('Cabin',axis=1)
clean_test_data=clean_test_data.drop('Cabin',axis=1)


In [83]:
#Now we have our clean data.
#And we can perform one-hard encoding on Sex and Embarked.

X_train=clean_train_data.drop('Survived',axis=1)
y_train=train_data['Survived']

for i in conts:
    y_train=y_train.drop(i)

X_test=clean_test_data

In [84]:
Object_cols=['Sex','Embarked']
clean_train_data['Embarked'].value_counts()

#Embarked has only three values，and no have NAN.

S    554
C    130
Q     28
Name: Embarked, dtype: int64

In [85]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False) #避免輸出稀疏矩陣
#When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will be all zeros.

OH_cols_train= pd.DataFrame(OH_encoder.fit_transform(X_train[Object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[Object_cols]))
OH_cols_train.index = X_train.index #We put indexes back
OH_cols_test.index = X_test.index #We put indexes back
#Now we drop the categorical columns
num_X_train = X_train.drop(Object_cols, axis=1)
num_X_test = X_test.drop(Object_cols, axis=1)
#And now we add the new columns
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)


OH_X_test.head()
print(OH_X_train.isnull().sum(axis = 0))

Pclass    0
Age       0
SibSp     0
Parch     0
Fare      0
0         0
1         0
2         0
3         0
4         0
5         0
dtype: int64


In [ ]:
#Now our Data is organized, next we use Machine Learning Model : Random Forest model.

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()

imputed_X_test = pd.DataFrame(my_imputer.fit_transform(OH_X_test))

#imputed_X_train.columns = X_train.columns
imputed_X_test.columns = OH_X_test.columns

In [87]:
#Analysis "Name" may benefit the model, 
#we can read take the original data an delete the some rows as like as "age".

names=train_data['Name']
names=names.drop(conts)
names['Name']=names


combine=[OH_X_train,imputed_X_test]


for dataset in combine:
    dataset['Title'] = names.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
#[A-Za-z]represents any letter
# "+" means the character before the "+"" sign appears one or more times.
# "\." represent "."
#([A-Za-z]+)\. is all combinations of strings that match one or more letters and end with "."

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    

#mapping
title_mapping = {"Mr": 1, "Rare": 2, "Master": 3, "Miss": 4, "Mrs": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

OH_X_train.head()

,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4,5,Title
0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0,0.0,1
1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0,5
2,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0,0.0,4
3,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0,0.0,5
4,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0,0.0,1


In [ ]:
#Finally, use random forest to achieve

from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier(n_estimators=1000, max_depth=5 , random_state=1)
model.fit(OH_X_train,y_train)
predictions=model.predict(imputed_X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)

Note : This data set is a project I saw at the beginning of my learn. I watched a lot of online video imitations, but the results of the kaggle score were not very good. 

After a while, I suddenly wanted to try it again without adding too many features, and the results were unexpectedly good. ,

I think the articles I have read in the past have caused "overfitting" results,
So data visualization and feature engineering is the part I want to strengthen.
